This exercise will require you to pull some data from the Qunadl API. Qaundl is currently the most widely used aggregator of financial market data.

As a first step, you will need to register a free account on the http://www.quandl.com website.

After you register, you will be provided with a unique API key, that you should store:

In [1]:
# Store the API key as a string - according to PEP8, constants are always named in all upper case
API_KEY = 'vGmSYi6TqbbxM2d1Ja8z'

Qaundl has a large number of data sources, but, unfortunately, most of them require a Premium subscription. Still, there are also a good number of free datasets.

For this mini project, we will focus on equities data from the Frankfurt Stock Exhange (FSE), which is available for free. We'll try and analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.

You can find the detailed Quandl API instructions here: https://docs.quandl.com/docs/time-series

While there is a dedicated Python package for connecting to the Quandl API, we would prefer that you use the *requests* package, which can be easily downloaded using *pip* or *conda*. You can find the documentation for the package here: http://docs.python-requests.org/en/master/ 

Finally, apart from the *requests* package, you are encouraged to not use any third party Python packages, such as *pandas*, and instead focus on what's available in the Python Standard Library (the *collections* module might come in handy: https://pymotw.com/3/collections/).
Also, since you won't have access to DataFrames, you are encouraged to us Python's native data structures - preferably dictionaries, though some questions can also be answered using lists.
You can read more on these data structures here: https://docs.python.org/3/tutorial/datastructures.html

Keep in mind that the JSON responses you will be getting from the API map almost one-to-one to Python's dictionaries. Unfortunately, they can be very nested, so make sure you read up on indexing dictionaries in the documentation provided above.

In [48]:
# First, import the relevant modules

import requests
import json
import statistics 
import numpy

In [6]:
# Now, call the Quandl API and pull out a small sample of the data (only one day) to get a glimpse
# into the JSON structure that will be returned
url = f"https://www.quandl.com/api/v3/datasets/FSE/AFX_X.json?&start_date=2017-01-01&end_date=2017-12-30&api_key={API_KEY}"
res = requests.get(url)

In [7]:
# Inspect the JSON structure of the object you created, and take note of how nested it is,
# as well as the overall structure
res_json = res.json() 

These are your tasks for this mini project:

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
2. Convert the returned JSON object into a Python dictionary.
3. Calculate what the highest and lowest opening prices were for the stock in this period.
4. What was the largest change in any one day (based on High and Low price)?
5. What was the largest change between any two days (based on Closing Price)?
6. What was the average daily trading volume during this year?
7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

In [58]:

HighestDate = ""
HighestPrice = 0
LowestDate = ""
LowestPrice = 9999
LargestChangeDate = ""
LargestChangePrice = 0
PrevDayOpenPrice = False
Largest2DayChangeDate = ""
Largest2DayChangePrice = 0
TotalTradeVolume = 0
TotalTradingDays = 0
for day_data in res_json["dataset"]["data"]:
    Date = day_data[0]
    Open = day_data[1]
    High = day_data[2]
    Low = day_data[3]
    Close = day_data[4]
    TradedVolume = day_data[6]
    Turnover = day_data[7]
    if(isinstance(Open, float) and isinstance(Close, float)):
        DailyPriceChange = abs(Open - Close)
        if(PrevDayOpenPrice == False):
            PrevDayOpenPrice = Open
        TwoDayPriceChange = abs(PrevDayOpenPrice - Close)
        PrevDayOpenPrice = Open
    else:
        DailyPriceChange = 0
        TwoDayPriceChange = 0
    
    TotalTradeVolume += TradedVolume
    TotalTradingDays += 1
    if(Low < LowestPrice):
        LowestDate = Date
        LowestPrice = Low
    if(High > HighestPrice):
        HighestDate = Date
        HighestPrice = High
    if(DailyPriceChange > LargestChangePrice):
        LargestChangeDate = Date
        LargestChangePrice = DailyPriceChange
    if(TwoDayPriceChange > Largest2DayChangePrice):
        Largest2DayChangeDate = Date
        Largest2DayChangePrice = TwoDayPriceChange
LargestChangePrice = round(LargestChangePrice, 2)
Largest2DayChangePrice = round(Largest2DayChangePrice, 2)
print(f"Highest was on {HighestDate} for {HighestPrice}")
print(f"Lowest was on {LowestDate} for {LowestPrice}")
print(f"Largest Daily Change was on {LargestChangeDate} for {LargestChangePrice}")
print(f"Largest 2 Day Change was on {Largest2DayChangeDate} for {Largest2DayChangePrice}")
print(f"Total Traded Volume was {TotalTradeVolume} in {TotalTradingDays} days ")

# use numpy and statistic to calculate median
num = numpy.array(res_json["dataset"]["data"])
volumes = num[:, 6]
median = statistics.median(volumes)
print(f"The Median Trading Volume was {median}") 

Highest was on 2017-12-14 for 53.54
Lowest was on 2017-01-23 for 33.62
Largest Daily Change was on 2017-08-09 for 1.69
Largest 2 Day Change was on 2017-03-21 for 2.73
Total Traded Volume was 22726706.0 in 255 days 
The Median Trading Volume was 76286.0
